In [1]:
import pandas as pd

In [8]:
# 현재 작업 디렉토리 확인 및 파일 검색
import os, glob
from pathlib import Path

print("cwd:", os.getcwd())
print("notebook path (resolved):", Path().resolve())
print("\nMatched files:")
for p in glob.glob("24.12*kospi*.csv"):
    print("-", p)

# 경로가 다른 경우 예: 상위 폴더에 있다면 ../를 붙여서 읽기
# df = pd.read_csv('../24.12_kospi200.csv', encoding='cp949')

# 파일 존재 여부 확인 예시
target = "24.12_kospi200 -회사명,종목코드.csv"
print("\nexists:", os.path.exists(target))

cwd: c:\Users\shcho\OneDrive\바탕 화면\최종 프로젝트\추가자료\코드
notebook path (resolved): C:\Users\shcho\OneDrive\바탕 화면\최종 프로젝트\추가자료\코드

Matched files:

exists: False


In [17]:
from pykrx import stock
import pandas as pd
from tqdm import tqdm
import time

In [22]:
df_k200 = pd.read_csv('24.12_kospi200 -회사명,종목코드.csv', dtype={'종목코드' : str})
tickers = df_k200["종목코드"]

In [23]:
df_k200

,종목코드,종목명
0,005930,삼성전자
1,000660,SK하이닉스
2,373220,LG에너지솔루션
3,207940,삼성바이오로직스
4,005380,현대차
...,...,...
195,178920,PI첨단소재
196,006110,삼아알미늄
197,005420,코스모화학
198,271940,일진하이솔루스


In [24]:
tickers

0      005930
1      000660
2      373220
3      207940
4      005380
        ...  
195    178920
196    006110
197    005420
198    271940
199    006650
Name: 종목코드, Length: 200, dtype: object

In [ ]:
start = "20150101"
end   = "20241231"


all_val_data = []

for ticker in tqdm(tickers):
    try:
        # ② 투자자별 '순매수 거래대금(원)' 데이터 불러오기
        df_val = stock.get_market_trading_value_by_date(
            fromdate=start,
            todate=end,
            ticker=ticker
        )

        # ③ 필요한 컬럼만 추출
        val = df_val[["기관합계", "개인", "외국인합계"]].copy()
        val.columns = ["기관_순매수_금액", "개인_순매수_금액", "외국인_순매수_금액"]

        # ④ ticker 정보 추가
        val["ticker"] = ticker

        all_val_data.append(val)
        time.sleep(3)

    except Exception as e:
        print(f"Error in {ticker}: {e}")

# ⑤ 모든 종목 데이터 합치기
result = pd.concat(all_val_data)
result = result.sort_index()
result.index.name = "date"

print(result.head())
print(result.tail())


 19%|█▉        | 38/200 [11:19<48:36, 18.00s/it] 

In [27]:
all_val_data

[              기관_순매수_금액     개인_순매수_금액    외국인_순매수_금액  ticker
 날짜                                                         
 2015-01-02 -27448674000   -7894072000   28661116000  005930
 2015-01-05  12453321000   -4283380000  -70494157000  005930
 2015-01-06  26776551000   39507501000 -133456517000  005930
 2015-01-07  28976493000   -4469805000  -78013040000  005930
 2015-01-08    133445000    6810295000   -5538895000  005930
 ...                 ...           ...           ...     ...
 2024-12-23  24961655300  -37622295300  -44488165300  005930
 2024-12-24   3599664600 -168358856200  103093057000  005930
 2024-12-26  -6930760500   13274611300  -73581477500  005930
 2024-12-27 -13922536000  -39507459700   -2171267300  005930
 2024-12-30  -3277287800   72859108500 -143037494300  005930
 
 [2455 rows x 4 columns],
               기관_순매수_금액     개인_순매수_금액    외국인_순매수_금액  ticker
 날짜                                                         
 2015-01-02  -9531805200   11670794600   -2222509500  000